### N-Sequence Loss Strategy
Objective: Bet on rounds once n losses have occurred. Idea that after a sequence of losses a win must occur. 

Minimise downside by adding stop_loss protection after m games lost.

In [2]:
import numpy as np
import hashlib
import random
import string
import hmac
import pandas as pd
from tqdm.notebook import tqdm
import os
import re

import config
import get_game_data

tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

### Load data

In [5]:
multipliers = get_game_data.get_all_multipliers(config.first_game,config.current_game)
multipliers_chronological_order = multipliers[::-1]
multiplier_df = pd.DataFrame({'game_no': range(1,len(multipliers_chronological_order)+1), 'multiplier': multipliers_chronological_order})

multipler_dataset_name = "raw_multipler_" + ".csv"
raw_multipler_data_path = os.path.join("data",multipler_dataset_name)
multiplier_df.to_csv(raw_multipler_data_path,index=False)

In [7]:
multipler_dataset_name = "raw_multipler_" + ".csv"
raw_multipler_data_path = os.path.join("data",multipler_dataset_name)
raw_multiplier_df = pd.read_csv(raw_multipler_data_path)
training_df = raw_multiplier_df.copy()

In [8]:
#create results folder
experiment_results_directory = "loss_sequence_playing"
results_path = os.path.join("data",experiment_results_directory)
try:    
    os.mkdir(results_path)
except FileExistsError:
    pass

### Multiple loss play
After multiple red games, a green game must occur.

In [9]:
#experiment parameters
loss_level = 2
auto_cashout_level = 1.5
bet_size = 0.1
num_loss = 4 #number of losses in a row
loss_counter = 0

In [10]:
training_df["loss_flag"] = False
training_df.loc[training_df["multiplier"] < loss_level, "loss_flag"] = True

training_df["cumsum"] = (~training_df["loss_flag"]).cumsum()

In [59]:
#find where cumsum doesn't change for num_games lost in a row
cumsum_value_counts = training_df["cumsum"].value_counts()
appropriate_cumsum = cumsum_value_counts[cumsum_value_counts >= num_loss+1]
game_sequences = training_df[training_df["cumsum"].isin(appropriate_cumsum.index)].query("loss_flag == True")

final_sequence_game_no = game_sequences.groupby(["cumsum"])["game_no"].last().reset_index()
final_sequence_game_no["game_no"] += 1
additional_games = training_df[training_df["game_no"].isin(final_sequence_game_no["game_no"])]
additional_games["cumsum"] -= 1
game_sequences = game_sequences.append(additional_games, ignore_index=True)

game_sequences = game_sequences.sort_values(by="game_no",ignore_index=True)


In [100]:
#run algorithm over game sequences
prev_game_no = 0
betting_results = []
all_game_batches = []
for game_no in tqdm(game_sequences["game_no"]):
    game = game_sequences.query("game_no == @game_no")
    if prev_game_no+1 != game_no:
        #new game sequence
        game_cumsum = game["cumsum"].values[0]
        sequence = game_sequences.query("cumsum == @game_cumsum")
        #split sequence into 
        num_batches = np.ceil(len(sequence.index)/(num_loss + 1))
        batch_prefixes = [0] + [idx*(num_loss+1) for idx in range(1,int(num_batches))]
        batch_suffixes = [(idx*num_loss)+1 for idx in range(1,int(num_batches))] + [len(sequence.index)]
        game_batches = [sequence.iloc[batch_prefixes[num_round]:batch_suffixes[num_round]] for num_round in range(len(batch_prefixes))]

        for batch in game_batches:
            all_game_batches.append(batch)
            total_losses = batch.iloc[:num_loss]["loss_flag"].sum()
            if num_loss == total_losses:
                bet_round = batch.iloc[-1]
                loss_flag = bet_round["loss_flag"]
                if not(loss_flag):
                    #round won
                    betting_result = bet_size * auto_cashout_level
                else:
                    betting_result = 0

                betting_results.append(betting_result)
        


  0%|          | 0/404438 [00:00<?, ?it/s]